### Import Libraries

In [1]:
import pandas as pd
import sqlalchemy, os
from binance import BinanceSocketManager
from binance.client import Client
from dotenv import load_dotenv
load_dotenv()

True

### Environment variables

In [2]:
API_KEY = os.environ.get("API_KEY")
API_SECRET = os.environ.get("API_SECRET")

In [4]:
# Binance Client
client = Client(API_KEY, API_SECRET)

In [5]:
# Creating Socket Manager
bsm = BinanceSocketManager(client)

In [6]:
# Creating a socket for BTC-USDT pair
socket = bsm.trade_socket('BTCUSDT')

In [7]:
# SQLite EngineSQLite Engine
engine = sqlalchemy.create_engine('sqlite:///BTCUSDTstream.db')

### Function for creating dataframe

In [8]:
def createframe(msg):
    df = pd.DataFrame([msg])
    df = df.loc[:,['s','E','p']]
    df.columns = ['symbol','Time','Price']
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time , unit='ms')
    return df

In [ ]:
while True:
    await socket.__aenter__()
    msg = await socket.recv()
    frame = createframe(msg)
    frame.to_sql('BTCUSDT', engine, if_exists='append', index=False)
    print(frame)

    symbol                    Time     Price
0  BTCUSDT 2022-02-04 09:07:41.732  37949.99
    symbol                    Time     Price
0  BTCUSDT 2022-02-04 09:07:41.732  37949.99
